In [1]:
# Token-Based Tendering System - Multi-Scenario Analysis
# Enhanced prototype demonstrating various procurement scenarios

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import json
import sys
import os
from collections import defaultdict

# Add utils directory to path
sys.path.append('utils')
from token_engine import TokenEngine

# Configuration
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("🏛️ Token-Based Tendering System - Multi-Scenario Analysis")
print("=" * 60)
print("Advanced prototype demonstrating various procurement scenarios")
print()

## Load and Prepare Data
tenders_df = pd.read_csv('data/tenders.csv')
contractors_df = pd.read_csv('data/contractors.csv')

# Initialize clean token engine for scenarios
engine = TokenEngine('data/transactions.json')

## Scenario Analysis Framework

class ScenarioRunner:
    """Run different procurement scenarios to test the token system"""
    
    def __init__(self, engine):
        self.engine = engine
        self.scenarios = {}
        
    def create_scenario(self, name, tender_idx, contractor_idx, 
                       quality_scores, spending_efficiency=1.0):
        """Create a procurement scenario"""
        tender = tenders_df.iloc[tender_idx]
        contractor = contractors_df.iloc[contractor_idx]
        
        scenario = {
            'name': name,
            'tender': tender,
            'contractor': contractor,
            'quality_scores': quality_scores,
            'spending_efficiency': spending_efficiency,
            'results': {}
        }
        
        self.scenarios[name] = scenario
        return scenario
    
    def run_scenario(self, scenario_name):
        """Execute a complete scenario"""
        scenario = self.scenarios[scenario_name]
        tender = scenario['tender']
        contractor = scenario['contractor']
        
        print(f"🎬 Running Scenario: {scenario['name']}")
        print(f"Project: {tender['project_name']}")
        print(f"Contractor: {contractor['company_name']}")
        print(f"Value: R{tender['total_value']:,}")
        
        # Parse project details
        project_scope = tender['project_scope'].split(';')
        milestones = tender['milestones'].split(';')
        
        # Issue tokens
        tender_id = f"{tender['tender_id']}_{scenario_name}"
        issuance = self.engine.issue_tokens(
            tender_id=tender_id,
            contractor_id=contractor['contractor_id'],
            total_value=tender['total_value'],
            project_scope=project_scope
        )
        
        # Execute milestones with varying spending patterns
        milestone_results = []
        total_spent = 0
        
        base_spending_per_milestone = tender['total_value'] / len(milestones)
        
        for i, (milestone, quality_score) in enumerate(zip(milestones, scenario['quality_scores'])):
            # Vary spending based on efficiency
            milestone_budget = base_spending_per_milestone * scenario['spending_efficiency']
            
            # Distribute spending across categories
            category_splits = [0.4, 0.3, 0.2, 0.1]  # Typical distribution
            
            milestone_spent = 0
            for j, (category, split) in enumerate(zip(project_scope, category_splits)):
                if j >= len(category_splits):
                    break
                    
                amount = milestone_budget * split
                
                try:
                    self.engine.spend_tokens(
                        tender_id=tender_id,
                        contractor_id=contractor['contractor_id'],
                        amount=amount,
                        category=category,
                        milestone=milestone,
                        description=f"{milestone} - {category}"
                    )
                    milestone_spent += amount
                except Exception as e:
                    print(f"   ⚠️ Spending error: {e}")
                    break
            
            total_spent += milestone_spent
            
            # Verify milestone
            verification = self.engine.verify_milestone(
                tender_id=tender_id,
                milestone=milestone,
                quality_score=quality_score
            )
            
            milestone_results.append({
                'milestone': milestone,
                'spent': milestone_spent,
                'quality_score': quality_score,
                'passed': verification['passed']
            })
            
            print(f"   ✅ {milestone}: {quality_score}% quality, R{milestone_spent:,.0f} spent")
        
        # Attempt redemption
        try:
            redemption = self.engine.redeem_tokens(
                tender_id=tender_id,
                contractor_id=contractor['contractor_id']
            )
            outcome = "REDEEMED"
            final_value = redemption['cash_value']
            print(f"   🎉 Tokens redeemed: R{redemption['tokens_redeemed']:,.0f} → R{final_value:,.0f}")
        except Exception as e:
            returned = self.engine.return_tokens_to_treasury(
                tender_id=tender_id,
                contractor_id=contractor['contractor_id']
            )
            outcome = "RETURNED"
            final_value = 0
            print(f"   ❌ Tokens returned to treasury: R{returned['tokens_returned']:,.0f}")
        
        # Store results
        scenario['results'] = {
            'tender_id': tender_id,
            'total_spent': total_spent,
            'milestone_results': milestone_results,
            'outcome': outcome,
            'final_value': final_value,
            'avg_quality': np.mean(scenario['quality_scores']),
            'efficiency': (tender['total_value'] - total_spent) / tender['total_value']
        }
        
        print(f"   📊 Average Quality: {scenario['results']['avg_quality']:.1f}%")
        print(f"   💰 Efficiency: {scenario['results']['efficiency']:.1%}")
        print()
        
        return scenario['results']

## Create Multiple Scenarios

runner = ScenarioRunner(engine)

# Scenario 1: Excellent Performance
runner.create_scenario(
    name="Excellent Performance",
    tender_idx=0,  # Johannesburg Civic Centre
    contractor_idx=0,  # Ubuntu Construction
    quality_scores=[95, 92, 90, 88],
    spending_efficiency=0.85  # 15% under budget
)

# Scenario 2: Poor Performance
runner.create_scenario(
    name="Poor Performance", 
    tender_idx=1,  # Cape Town Library
    contractor_idx=4,  # Eastern Cape Development (lower rating)
    quality_scores=[75, 70, 65, 72],
    spending_efficiency=1.1  # 10% over budget
)

# Scenario 3: Mixed Performance
runner.create_scenario(
    name="Mixed Performance",
    tender_idx=2,  # Durban Sports Complex
    contractor_idx=2,  # Coastal Engineering (high rating)
    quality_scores=[85, 78, 92, 81],
    spending_efficiency=0.95  # 5% under budget
)

# Scenario 4: Exceptional Efficiency
runner.create_scenario(
    name="Exceptional Efficiency",
    tender_idx=3,  # Pretoria School Renovation
    contractor_idx=1,  # Proudly SA Builders
    quality_scores=[88, 94, 91, 89],
    spending_efficiency=0.75  # 25% under budget
)

## Run All Scenarios

print("🎭 RUNNING MULTIPLE PROCUREMENT SCENARIOS")
print("=" * 50)

all_results = {}
for scenario_name in runner.scenarios.keys():
    all_results[scenario_name] = runner.run_scenario(scenario_name)

## Comparative Analysis

print("📊 COMPARATIVE SCENARIO ANALYSIS")
print("=" * 40)

# Create comparison dataframe
comparison_data = []
for name, results in all_results.items():
    scenario = runner.scenarios[name]
    comparison_data.append({
        'Scenario': name,
        'Project Value': scenario['tender']['total_value'],
        'Contractor Rating': scenario['contractor']['rating'],
        'Avg Quality Score': results['avg_quality'],
        'Spending Efficiency': results['efficiency'],
        'Final Outcome': results['outcome'],
        'Contractor Benefit': results['final_value'],
        'Government Savings': scenario['tender']['total_value'] - results['total_spent']
    })

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.round(2))
print()

## Advanced Visualizations

fig = plt.figure(figsize=(20, 15))

# 1. Scenario Comparison Matrix
ax1 = plt.subplot(2, 3, 1)
scatter = ax1.scatter(comparison_df['Avg Quality Score'], 
                     comparison_df['Spending Efficiency'],
                     s=comparison_df['Project Value']/1e6,  # Size by project value
                     c=comparison_df['Contractor Rating'],
                     cmap='RdYlGn', alpha=0.7)

ax1.set_xlabel('Average Quality Score (%)')
ax1.set_ylabel('Spending Efficiency')
ax1.set_title('Quality vs Efficiency by Scenario')
ax1.axhline(y=0, color='black', linestyle='--', alpha=0.5)
ax1.axvline(x=80, color='red', linestyle='--', alpha=0.5, label='Quality Threshold')

# Add scenario labels
for i, row in comparison_df.iterrows():
    ax1.annotate(row['Scenario'], 
                (row['Avg Quality Score'], row['Spending Efficiency']),
                xytext=(5, 5), textcoords='offset points', 
                fontsize=8, alpha=0.8)

plt.colorbar(scatter, ax=ax1, label='Contractor Rating')
ax1.legend()

# 2. Financial Outcomes
ax2 = plt.subplot(2, 3, 2)
outcomes = comparison_df['Final Outcome'].value_counts()
colors = ['green' if x == 'REDEEMED' else 'red' for x in outcomes.index]
ax2.bar(outcomes.index, outcomes.values, color=colors, alpha=0.7)
ax2.set_title('Token Redemption Outcomes')
ax2.set_ylabel('Number of Projects')

# 3. Government Savings Analysis
ax3 = plt.subplot(2, 3, 3)
savings = comparison_df['Government Savings'] / 1e6  # Convert to millions
colors = ['green' if x > 0 else 'red' for x in savings]
bars = ax3.bar(range(len(comparison_df)), savings, color=colors, alpha=0.7)
ax3.set_title('Government Savings by Scenario (R Millions)')
ax3.set_ylabel('Savings (R Millions)')
ax3.set_xticks(range(len(comparison_df)))
ax3.set_xticklabels(comparison_df['Scenario'], rotation=45, ha='right')

# Add value labels
for bar, saving in zip(bars, savings):
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height + (0.5 if height > 0 else -1),
             f'R{saving:.1f}M', ha='center', va='bottom' if height > 0 else 'top')

# 4. Quality Score Distribution
ax4 = plt.subplot(2, 3, 4)
all_quality_scores = []
scenario_labels = []
for name, results in all_results.items():
    scores = [m['quality_score'] for m in results['milestone_results']]
    all_quality_scores.extend(scores)
    scenario_labels.extend([name] * len(scores))

quality_df = pd.DataFrame({'Score': all_quality_scores, 'Scenario': scenario_labels})
sns.boxplot(data=quality_df, x='Scenario', y='Score', ax=ax4)
ax4.axhline(y=80, color='red', linestyle='--', alpha=0.7, label='Pass Threshold')
ax4.set_title('Quality Score Distribution by Scenario')
ax4.set_xticklabels(ax4.get_xticklabels(), rotation=45, ha='right')
ax4.legend()

# 5. Contractor Performance vs Rating
ax5 = plt.subplot(2, 3, 5)
ax5.scatter(comparison_df['Contractor Rating'], comparison_df['Avg Quality Score'], 
           s=100, alpha=0.7, c=comparison_df['Spending Efficiency'], cmap='RdYlGn')
ax5.set_xlabel('Contractor Rating (Pre-project)')
ax5.set_ylabel('Average Quality Score (Actual)')
ax5.set_title('Contractor Rating vs Actual Performance')

# Add trend line
z = np.polyfit(comparison_df['Contractor Rating'], comparison_df['Avg Quality Score'], 1)
p = np.poly1d(z)
ax5.plot(comparison_df['Contractor Rating'], p(comparison_df['Contractor Rating']), "r--", alpha=0.8)

# 6. System Effectiveness Summary
ax6 = plt.subplot(2, 3, 6)
metrics = ['Quality Control', 'Cost Efficiency', 'Transparency', 'Fraud Prevention']
traditional_scores = [3, 2, 2, 1]  # Out of 5
token_scores = [5, 4, 5, 5]  # Out of 5

x = np.arange(len(metrics))
width = 0.35

ax6.bar(x - width/2, traditional_scores, width, label='Traditional System', alpha=0.7)
ax6.bar(x + width/2, token_scores, width, label='Token System', alpha=0.7)

ax6.set_xlabel('System Aspects')
ax6.set_ylabel('Effectiveness Score (1-5)')
ax6.set_title('System Effectiveness Comparison')
ax6.set_xticks(x)
ax6.set_xticklabels(metrics, rotation=45, ha='right')
ax6.legend()
ax6.set_ylim(0, 5)

plt.tight_layout()
plt.show()

## System Impact Assessment

print("🎯 SYSTEM IMPACT ASSESSMENT")
print("=" * 35)

# Calculate overall system performance
total_projects = len(all_results)
successful_projects = len([r for r in all_results.values() if r['outcome'] == 'REDEEMED'])
total_value = sum(comparison_df['Project Value'])
total_savings = sum(comparison_df['Government Savings'])
avg_quality = comparison_df['Avg Quality Score'].mean()

print(f"📊 Overall System Performance:")
print(f"   • Total Projects Simulated: {total_projects}")
print(f"   • Successful Completions: {successful_projects}/{total_projects} ({successful_projects/total_projects:.1%})")
print(f"   • Total Project Value: R{total_value:,}")
print(f"   • Total Government Savings: R{total_savings:,}")
print(f"   • Average Quality Score: {avg_quality:.1f}%")
print(f"   • System Success Rate: {successful_projects/total_projects:.1%}")
print()

print("💡 Key Insights:")
insights = [
    f"✅ High-performing contractors (rating >4.0) achieved {comparison_df[comparison_df['Contractor Rating']>4.0]['Avg Quality Score'].mean():.1f}% avg quality",
    f"💰 Efficient contractors saved government avg R{comparison_df[comparison_df['Spending Efficiency']>0]['Government Savings'].mean()/1e6:.1f}M per project",
    f"⚠️ Poor performers had tokens worth R{sum(r['total_spent'] for r in all_results.values() if r['outcome']=='RETURNED')/1e6:.1f}M returned to treasury",
    f"🎯 Quality threshold (80%) effectively separated good from poor performers",
    f"📈 Token system incentivized {successful_projects}/{total_projects} contractors to meet quality standards"
]

for insight in insights:
    print(f"   {insight}")

print()

## Policy Recommendations

print("📋 POLICY RECOMMENDATIONS")
print("=" * 30)

recommendations = [
    "🏛️ IMPLEMENTATION STRATEGY:",
    "   • Start with projects R10M-R50M for pilot phase",
    "   • Require contractor insurance for token-based projects", 
    "   • Establish independent quality assessment panels",
    "",
    "⚖️ REGULATORY FRAMEWORK:",
    "   • Update Municipal Finance Management Act",
    "   • Create token system legal definitions",
    "   • Establish dispute resolution mechanisms",
    "",
    "🔧 TECHNICAL REQUIREMENTS:",
    "   • Real-time monitoring dashboard for officials",
    "   • Mobile app for contractor token management",
    "   • Integration with existing procurement systems",
    "",
    "👥 STAKEHOLDER ENGAGEMENT:",
    "   • Training programs for procurement officials",
    "   • Contractor education and certification",
    "   • Public awareness campaigns on system benefits"
]

for rec in recommendations:
    print(rec)

print()
print("🚀 Prototype simulation complete! Ready for pilot implementation.")

ModuleNotFoundError: No module named 'token_engine'